# Maxabi


Nous allons utiliser la base de données `test`, dans laquelle se trouve la collection `restaurants`. Celle-ci est composée de plus de 25000 retaurants new-yorkais, avec des évaluations de ceux-ci.

1. Créer une connexion vers cette collection
1. Déterminer le nombre exact de restaurants.
1. Lister les différents quartiers représentés (*borough*) ainsi que les différents types de cuisine (*cuisine*).
1. Lister les noms des restaurants, ainsi que leur quartier, ne proposant que des soupes (*cuisine* égale à *Soups*).
1. Donner les nombre de restaurants ayant eu au moins une fois le *grade* (dans *grades*) `Z`.
1. Calculer le nombre de restaurants pour chaque quartier
1. Donner les cinq types de cuisine les plus présentes dans New-York (avec le plus de restaurants donc)
1. Pour chaque *grade* possible, donner le nombre d'évaluation avec ce grade, ainsi que le *score* moyen, minimum et maximum.
1. Créer un `DataFrame` contenant les restaurants en ligne et en colonne le score moyen obtenu pour chaque *grade* possible. Si un restaurant n'a pas eu d'évaluation d'un *grade*, nous aurons une valeur absente.

In [3]:
# Imports
import pymongo
import pprint
import pandas as pd

**1. Créer une connexion vers cette collection**

In [4]:
# Connection
con = pymongo.MongoClient("mongodb://193.51.82.104:2343/")
db = con.test

**2. Déterminer le nombre exact de restaurants.**

In [5]:
# db.count()
db.restaurants.estimated_document_count()

25359

**3. Lister les différents quartiers représentés (borough) ainsi que les différents types de cuisine (cuisine).**

In [6]:
db.restaurants.distinct("borough")

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island', 'Missing']

In [7]:
db.restaurants.distinct("cuisine")

['Bakery',
 'Hamburgers',
 'Irish',
 'American',
 'Jewish/Kosher',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Chicken',
 'Turkish',
 'Caribbean',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'Bagels/Pretzels',
 'Continental',
 'Pizza',
 'Italian',
 'Steak',
 'Polish',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'German',
 'French',
 'Pizza/Italian',
 'Mexican',
 'Spanish',
 'Café/Coffee/Tea',
 'Tex-Mex',
 'Pancakes/Waffles',
 'Soul Food',
 'Seafood',
 'Hotdogs',
 'Greek',
 'Not Listed/Not Applicable',
 'African',
 'Japanese',
 'Indian',
 'Armenian',
 'Thai',
 'Chinese/Cuban',
 'Mediterranean',
 'Korean',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Russian',
 'Eastern European',
 'Middle Eastern',
 'Asian',
 'Ethiopian',
 'Vegetarian',
 'Barbecue',
 'Egyptian',
 'English',
 'Other',
 'Sandwiches',
 'Portuguese',
 'Indonesian',
 'Chinese/Japanese',
 'Filipino',
 'Juice, Smoothies, Fruit Salads',
 'Brazilian',
 'Afghan'

In [8]:
quartier_cuisine = db.restaurants.distinct("borough") + db.restaurants.distinct("cuisine")
quartier_cuisine

['Bronx',
 'Brooklyn',
 'Manhattan',
 'Queens',
 'Staten Island',
 'Missing',
 'Bakery',
 'Hamburgers',
 'Irish',
 'American',
 'Jewish/Kosher',
 'Delicatessen',
 'Ice Cream, Gelato, Yogurt, Ices',
 'Chinese',
 'Chicken',
 'Turkish',
 'Caribbean',
 'Donuts',
 'Sandwiches/Salads/Mixed Buffet',
 'Bagels/Pretzels',
 'Continental',
 'Pizza',
 'Italian',
 'Steak',
 'Polish',
 'Latin (Cuban, Dominican, Puerto Rican, South & Central American)',
 'German',
 'French',
 'Pizza/Italian',
 'Mexican',
 'Spanish',
 'Café/Coffee/Tea',
 'Tex-Mex',
 'Pancakes/Waffles',
 'Soul Food',
 'Seafood',
 'Hotdogs',
 'Greek',
 'Not Listed/Not Applicable',
 'African',
 'Japanese',
 'Indian',
 'Armenian',
 'Thai',
 'Chinese/Cuban',
 'Mediterranean',
 'Korean',
 'Bottled beverages, including water, sodas, juices, etc.',
 'Russian',
 'Eastern European',
 'Middle Eastern',
 'Asian',
 'Ethiopian',
 'Vegetarian',
 'Barbecue',
 'Egyptian',
 'English',
 'Other',
 'Sandwiches',
 'Portuguese',
 'Indonesian',
 'Chinese/Japa

**4. Lister les noms des restaurants, ainsi que leur quartier, ne proposant que des soupes (cuisine égale à Soups).**

In [9]:
def affiche(res):
    pprint.pprint(list(res))

In [10]:
res = db.restaurants.find(
    { "cuisine": "Soups" }, 
    { "_id": 0, 
     "name": 1, 
     "borough": 1}
)

affiche(res)

[{'borough': 'Manhattan', 'name': 'Wichcraft Express'},
 {'borough': 'Staten Island', 'name': 'Original Soupman Of Staten Island'},
 {'borough': 'Manhattan', 'name': 'The Original Soupman'},
 {'borough': 'Manhattan', 'name': 'Peasant Stock'}]


**5. Donner les nombre de restaurants ayant eu au moins une fois le grade (dans grades) Z.**

In [11]:
res = db.restaurants.aggregate([
    {"$match": {"grades.grade": "Z"}},
    {"$group": {"_id": None, "nb": {"$sum": 1}}} 
])

affiche(res)

[{'_id': None, 'nb': 1337}]


**6. Calculer le nombre de restaurants pour chaque quartier**

In [12]:
res = db.restaurants.aggregate([
    {"$group": {"_id": "$borough", 
                "nb": {"$sum": 1}}} 
])

affiche(res)

[{'_id': 'Missing', 'nb': 51},
 {'_id': 'Staten Island', 'nb': 969},
 {'_id': 'Brooklyn', 'nb': 6086},
 {'_id': 'Bronx', 'nb': 2338},
 {'_id': 'Queens', 'nb': 5656},
 {'_id': 'Manhattan', 'nb': 10259}]


**7. Donner les cinq types de cuisine les plus présentes dans New-York (avec le plus de restaurants donc)**

In [13]:
res = db.restaurants.aggregate([ 
    {"$group": {"_id": "$cuisine" , "nb" : {"$sum" : 1}}},
    {"$sort" : {"nb": -1}},
    {"$limit" : 5}
])
affiche(res)

[{'_id': 'American', 'nb': 6183},
 {'_id': 'Chinese', 'nb': 2418},
 {'_id': 'Café/Coffee/Tea', 'nb': 1214},
 {'_id': 'Pizza', 'nb': 1163},
 {'_id': 'Italian', 'nb': 1069}]


**8. Pour chaque grade possible, donner le nombre d'évaluation avec ce grade, ainsi que le score moyen, minimum et maximum**

In [14]:
res = db.restaurants.aggregate([
    {"$group" : { 
        "_id": "$grades.grade", 
        "nombre" : {"$sum" : 1 },
        "moyenne" : {"$avg" : "$grades.score"},
        "minimum" : {"$min" : "$grades.score"},
        "maximum" : {"$max" : "$grades.score"}
    }}
])
affiche(res)

[{'_id': ['Z', 'A', 'P'],
  'maximum': [44, 13, 10],
  'minimum': [30, 10, 2],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['C', 'P'],
  'maximum': [43, 4],
  'minimum': [30, 10],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['Z', 'P'],
  'maximum': [25, 5],
  'minimum': [3, 5],
  'moyenne': None,
  'nombre': 7},
 {'_id': ['B', 'A', 'P'],
  'maximum': [21, 13, 10],
  'minimum': [14, 5, 4],
  'moyenne': None,
  'nombre': 4},
 {'_id': ['C', 'B', 'P'],
  'maximum': [36, 20, 5],
  'minimum': [36, 20, 5],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'C', 'P'],
  'maximum': [30, 10, 28, 17],
  'minimum': [30, 10, 28, 17],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'P', 'C'],
  'maximum': [13, 18, 2, 31],
  'minimum': [13, 18, 2, 31],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'P', 'B', 'A'],
  'maximum': [19, 13, 2, 16, 10],
  'minimum': [19, 13, 2, 16, 10],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'C', 'A'],
  'maximum': [34, 5, 13],
  'minimum': [1

  'minimum': [15, 10, 28, 24, 11],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'A', 'A', 'A', 'P', 'B', 'A', 'B'],
  'maximum': [11, 10, 5, 10, 13, 23, 7, 19],
  'minimum': [11, 10, 5, 10, 13, 23, 7, 19],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'C', 'B', 'C', 'A'],
  'maximum': [15, 43, 16, 31, 9],
  'minimum': [15, 43, 16, 31, 9],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'A', 'B', 'A', 'B', 'A', 'B'],
  'maximum': [12, 12, 17, 11, 24, 13, 18],
  'minimum': [12, 12, 17, 11, 24, 13, 18],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'A', 'B', 'A', 'B', 'C'],
  'maximum': [18, 9, 7, 24, 6, 26, 30],
  'minimum': [18, 9, 7, 24, 6, 26, 30],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['B', 'P', 'A', 'B', 'B', 'B', 'B'],
  'maximum': [14, 9, 10, 22, 18, 25, 14],
  'minimum': [14, 9, 10, 22, 18, 25, 14],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'P', 'A', 'P', 'B', 'P', 'A'],
  'maximum': [13, 17, 0, 13, 2, 17, 9, 13],
  'minimum': [13, 17, 0, 13

  'maximum': [16, 21, 12, 29, 18, 12, 15],
  'minimum': [16, 21, 12, 29, 18, 12, 15],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'A', 'A', 'A', 'A', 'P', 'A', 'C'],
  'maximum': [11, 12, 9, 9, 11, 5, 10, 3],
  'minimum': [11, 12, 9, 9, 11, 5, 10, 3],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'A', 'B', 'C', 'C', 'A'],
  'maximum': [13, 15, 9, 22, 30, 30, 13],
  'minimum': [13, 15, 9, 22, 30, 30, 13],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'C', 'C', 'A', 'B'],
  'maximum': [13, 33, 3, 5, 14],
  'minimum': [11, 34, 59, 12, 26],
  'moyenne': None,
  'nombre': 3},
 {'_id': ['Z', 'A', 'C', 'A', 'B'],
  'maximum': [36, 12, 36, 12, 14],
  'minimum': [2, 13, 38, 10, 19],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['B', 'A', 'C', 'C', 'A'],
  'maximum': [14, 12, 34, 38, 13],
  'minimum': [14, 12, 34, 38, 13],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'A', 'B', 'A', 'P', 'A', 'A'],
  'maximum': [9, 10, 19, 5, 2, 11, 12],
  'minimum': [9, 10, 19, 5, 2, 11, 1

  'minimum': [12, 10, 4, 42, 27, 15],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'B', 'P', 'B', 'B', 'A'],
  'maximum': [32, 12, 16, 10, 23, 22, 13],
  'minimum': [32, 12, 16, 10, 23, 22, 13],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'A', 'B', 'A', 'A', 'A', 'P'],
  'maximum': [8, 11, 18, 12, 10, 10, 2],
  'minimum': [8, 11, 18, 12, 10, 10, 2],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'A', 'A', 'C', 'A'],
  'maximum': [23, 7, 12, 10, 28, 11],
  'minimum': [21, 11, 11, 12, 42, 11],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['Z', 'B', 'P', 'B', 'A', 'P', 'A', 'B'],
  'maximum': [40, 14, 2, 24, 10, 3, 13, 27],
  'minimum': [40, 14, 2, 24, 10, 3, 13, 27],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'A', 'A', 'A', 'B', 'C'],
  'maximum': [11, 17, 11, 10, 9, 22, 36],
  'minimum': [11, 17, 11, 10, 9, 22, 36],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['C', 'A', 'B', 'B', 'B', 'B'],
  'maximum': [51, 7, 18, 14, 26, 24],
  'minimum': [51, 7, 18,

 {'_id': ['Z', 'C', 'B', 'A', 'B'],
  'maximum': [19, 48, 16, 9, 18],
  'minimum': [19, 48, 16, 9, 18],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'P', 'B', 'B', 'B', 'B'],
  'maximum': [10, 2, 19, 23, 16, 17],
  'minimum': [10, 2, 19, 23, 16, 17],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'C', 'A', 'C', 'A'],
  'maximum': [13, 41, 13, 33, 13],
  'minimum': [11, 44, 11, 38, 5],
  'moyenne': None,
  'nombre': 4},
 {'_id': ['B', 'A', 'A', 'A', 'P', 'B', 'A'],
  'maximum': [15, 2, 11, 13, 5, 26, 8],
  'minimum': [15, 2, 11, 13, 5, 26, 8],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'B', 'C', 'B', 'B'],
  'maximum': [9, 21, 18, 44, 25, 24],
  'minimum': [9, 21, 18, 44, 25, 24],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['B', 'B', 'B', 'A', 'C', 'B'],
  'maximum': [27, 22, 19, 13, 34, 24],
  'minimum': [26, 18, 22, 7, 10, 17],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['Z', 'A'],
  'maximum': [71, 9],
  'minimum': [2, 12],
  'moyenne': None,
  'nombre': 100},
 

...

 {'_id': ['A', 'B', 'C', 'A', 'C', 'B'],
  'maximum': [12, 20, 28, 4, 2, 19],
  'minimum': [10, 23, 30, 13, 31, 22],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['A', 'A', 'C', 'A', 'B', 'A'],
  'maximum': [13, 13, 4, 5, 23, 10],
  'minimum': [5, 9, 40, 12, 14, 11],
  'moyenne': None,
  'nombre': 10},
 {'_id': ['Z', 'B', 'B', 'B', 'A'],
  'maximum': [51, 15, 14, 21, 13],
  'minimum': [17, 20, 22, 24, 7],
  'moyenne': None,
  'nombre': 10},
 {'_id': ['A', 'A', 'C', 'A', 'B', 'B'],
  'maximum': [13, 10, 3, 11, 25, 17],
  'minimum': [11, 12, 53, 12, 22, 23],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['B', 'A', 'A', 'B'],
  'maximum': [26, 12, 9, 19],
  'minimum': [15, 7, 13, 21],
  'moyenne': None,
  'nombre': 25},
 {'_id': ['C', 'P', 'A', 'B', 'A'],
  'maximum': [35, 5, 10, 22, 9],
  'minimum': [35, 5, 10, 22, 9],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'B', 'B', 'P', 'B'],
  'maximum': [12, 25, 26, 25, 5, 24],
  'minimum': [12, 25, 26, 25, 5, 24],
  'moyenne': None,
  '

  'nombre': 1},
 {'_id': ['A', 'A', 'B', 'C', 'B'],
  'maximum': [12, 11, 16, 33, 17],
  'minimum': [5, 13, 19, 36, 24],
  'moyenne': None,
  'nombre': 6},
 {'_id': ['A', 'A', 'B', 'B', 'C', 'P', 'B'],
  'maximum': [9, 2, 20, 26, 13, 4, 27],
  'minimum': [9, 2, 20, 26, 13, 4, 27],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'C', 'A', 'C'],
  'maximum': [10, 22, 39, 4, 50],
  'minimum': [10, 22, 39, 4, 50],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['C', 'A', 'A'],
  'maximum': [89, 6, 13],
  'minimum': [28, 9, 12],
  'moyenne': None,
  'nombre': 17},
 {'_id': ['A', 'A', 'A', 'B', 'A', 'B', 'B'],
  'maximum': [12, 12, 4, 22, 7, 14, 17],
  'minimum': [9, 10, 12, 18, 9, 24, 19],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['A', 'C', 'P', 'C', 'B', 'A', 'B'],
  'maximum': [8, 11, 2, 46, 14, 7, 17],
  'minimum': [8, 11, 2, 46, 14, 7, 17],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'A', 'A', 'A', 'A', 'B'],
  'maximum': [19, 8, 13, 11, 2, 12, 26],
  'minimum': [19, 8

  'nombre': 4},
 {'_id': ['A', 'C'],
  'maximum': [13, 54],
  'minimum': [2, 2],
  'moyenne': None,
  'nombre': 63},
 {'_id': ['A', 'B', 'C', 'B', 'B', 'B'],
  'maximum': [10, 20, 39, 26, 25, 19],
  'minimum': [4, 16, 31, 27, 19, 15],
  'moyenne': None,
  'nombre': 3},
 {'_id': ['B', 'A', 'A', 'A', 'P'],
  'maximum': [22, 13, 12, 10, 3],
  'minimum': [15, 11, 9, 4, 12],
  'moyenne': None,
  'nombre': 3},
 {'_id': ['Z', 'A', 'B', 'C', 'A'],
  'maximum': [27, 8, 17, 34, 13],
  'minimum': [18, 12, 25, 35, 12],
  'moyenne': None,
  'nombre': 4},
 {'_id': ['A', 'A', 'B', 'B', 'C'],
  'maximum': [13, 12, 16, 17, 30],
  'minimum': [7, 10, 15, 25, 29],
  'moyenne': None,
  'nombre': 3},
 {'_id': ['A', 'C', 'B', 'A', 'A', 'C'],
  'maximum': [10, 51, 22, 13, 13, 2],
  'minimum': [9, 31, 21, 12, 5, 36],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['A', 'B', 'B', 'A', 'B', 'C', 'P'],
  'maximum': [7, 20, 24, 12, 21, 43, 9],
  'minimum': [7, 20, 24, 12, 21, 43, 9],
  'moyenne': None,
  'nombre': 1}

  'moyenne': None,
  'nombre': 2326},
 {'_id': ['A', 'A', 'A', 'B', 'C', 'C'],
  'maximum': [12, 12, 13, 20, 37, 35],
  'minimum': [7, 7, 13, 18, 9, 31],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['B', 'A', 'B', 'C', 'P', 'A'],
  'maximum': [24, 12, 19, 29, 10, 12],
  'minimum': [24, 12, 19, 29, 10, 12],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'C', 'A', 'A'],
  'maximum': [40, 7, 5, 9, 8],
  'minimum': [17, 12, 52, 10, 9],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['A', 'A', 'B', 'P', 'B', 'B', 'A'],
  'maximum': [12, 3, 20, 9, 16, 16, 11],
  'minimum': [12, 3, 20, 9, 16, 16, 11],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'C', 'B', 'P', 'B'],
  'maximum': [13, 22, 0, 21, 0, 27],
  'minimum': [13, 22, 0, 21, 0, 27],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'P', 'A', 'A', 'A', 'C'],
  'maximum': [5, 14, 3, 10, 7, 11, 34],
  'minimum': [5, 14, 3, 10, 7, 11, 34],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'B', 'B', 'B', 'A'],
  'maximum'

  'minimum': [12, 7, 18, 15, 24, 3, 17, 21],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['B', 'B', 'B', 'B', 'B', 'A'],
  'maximum': [20, 26, 23, 17, 14, 9],
  'minimum': [19, 15, 25, 15, 27, 13],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['A', 'A', 'A', 'B', 'A', 'C', 'A'],
  'maximum': [9, 10, 13, 20, 11, 6, 5],
  'minimum': [9, 10, 13, 20, 11, 6, 5],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'A', 'A', 'C', 'B'],
  'maximum': [42, 10, 13, 5, 21],
  'minimum': [33, 7, 12, 3, 16],
  'moyenne': None,
  'nombre': 2},
 {'_id': ['A', 'A', 'A', 'A', 'C', 'A', 'C'],
  'maximum': [4, 11, 2, 2, 55, 13, 4],
  'minimum': [4, 11, 2, 2, 55, 13, 4],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['Z', 'C', 'A', 'A', 'C'],
  'maximum': [23, 7, 13, 8, 42],
  'minimum': [23, 7, 13, 8, 42],
  'moyenne': None,
  'nombre': 1},
 {'_id': ['A', 'A', 'A', 'A', 'A', 'A', 'P'],
  'maximum': [12, 10, 11, 10, 13, 7, 10],
  'minimum': [2, 13, 11, 6, 13, 13, 6],
  'moyenne': None,
  'nombre': 5},
 {'_id': 

In [15]:
res = db.restaurants.aggregate([
    {"$unwind" : "$grades"},
    {"$group" : { 
        "_id": "$grades.grade", 
        "nombre" : {"$sum" : 1 },
        "moyenne" : {"$avg" : "$grades.score"},
        "minimum" : {"$min" : "$grades.score"},
        "maximum" : {"$max" : "$grades.score"}
    }}
])
affiche(res)

[{'_id': 'P',
  'maximum': 43,
  'minimum': 0,
  'moyenne': 5.506265664160401,
  'nombre': 1197},
 {'_id': 'A',
  'maximum': 27,
  'minimum': 0,
  'moyenne': 9.021954029147022,
  'nombre': 74656},
 {'_id': 'Not Yet Graded',
  'maximum': 75,
  'minimum': 0,
  'moyenne': 15.853515625,
  'nombre': 525},
 {'_id': 'B',
  'maximum': 57,
  'minimum': -1,
  'moyenne': 20.002459731809886,
  'nombre': 12603},
 {'_id': 'Z',
  'maximum': 71,
  'minimum': -1,
  'moyenne': 23.702318623784592,
  'nombre': 1337},
 {'_id': 'C',
  'maximum': 131,
  'minimum': -1,
  'moyenne': 30.70079491255962,
  'nombre': 3145}]


**9. Créer un `DataFrame` contenant les restaurants en ligne et en colonne le score moyen obtenu pour chaque grade possible. Si un restaurant n'a pas eu d'évaluation d'un grade, nous aurons une valeur absente.**

Si nous regardons le nombre de restaurants par nom de ceux-ci, nous remarquons qu'il y a des restaurants avec le même nom (des chaînes comme *Starbucks* ou *Subway* par exemple).

In [18]:
res = db.restaurants.aggregate([
    { "$group": { "_id": "$name", "nb": { "$sum": 1 } } },
    { "$sort": { "nb": -1 } },
    { "$limit": 10 }
])
affiche(res)

[{'_id': 'Subway', 'nb': 421},
 {'_id': 'Starbucks Coffee', 'nb': 223},
 {'_id': "Mcdonald'S", 'nb': 208},
 {'_id': "Dunkin' Donuts", 'nb': 206},
 {'_id': 'Dunkin Donuts', 'nb': 200},
 {'_id': '', 'nb': 151},
 {'_id': 'Kennedy Fried Chicken', 'nb': 88},
 {'_id': 'Crown Fried Chicken', 'nb': 81},
 {'_id': 'Burger King', 'nb': 80},
 {'_id': "Domino'S Pizza", 'nb': 67}]


Nous allons donc faire un regroupement par identifiant de restaurant (`restaurant_id`) et par `grade`, pour calculer le score moyen. Nous voyons ici les premiers résultats de l'aggrégation.

In [19]:
res = db.restaurants.aggregate([
    { "$unwind" : "$grades"},
    { "$group" : { 
        "_id": { "restaurant_id": "$restaurant_id", "grade": "$grades.grade" }, 
        "moy" : { "$avg" : "$grades.score" }
    } }
])
l = list(res)
l[:10]

[{'_id': {'restaurant_id': '50018661', 'grade': 'Not Yet Graded'}, 'moy': 2.0},
 {'_id': {'restaurant_id': '50018622', 'grade': 'Not Yet Graded'},
  'moy': 19.0},
 {'_id': {'restaurant_id': '50018608', 'grade': 'A'}, 'moy': 12.0},
 {'_id': {'restaurant_id': '50018587', 'grade': 'Not Yet Graded'}, 'moy': 2.0},
 {'_id': {'restaurant_id': '50018565', 'grade': 'Not Yet Graded'}, 'moy': 0.0},
 {'_id': {'restaurant_id': '50018547', 'grade': 'Not Yet Graded'},
  'moy': 29.0},
 {'_id': {'restaurant_id': '50018535', 'grade': 'Not Yet Graded'},
  'moy': 47.0},
 {'_id': {'restaurant_id': '50018514', 'grade': 'Not Yet Graded'}, 'moy': 9.0},
 {'_id': {'restaurant_id': '50018480', 'grade': 'Not Yet Graded'},
  'moy': 43.0},
 {'_id': {'restaurant_id': '50018450', 'grade': 'Not Yet Graded'}, 'moy': 0.0}]

Ensuite, on créé un `DataFrame` *à la main*, en récupérant la liste des identifiants, la liste des grades et la liste des scores moyens. Ce qui nous donne à la fin le `DataFrame` suivant.

In [21]:
df = pd.DataFrame({
    "restaurant_id"  : [r["_id"]["restaurant_id"] for r in l],
    "grade": [r["_id"]["grade"] for r in l],
    "score": [r["moy"] for r in l]
})
df.head()

,restaurant_id,grade,score
0,50018661,Not Yet Graded,2.0
1,50018622,Not Yet Graded,19.0
2,50018608,A,12.0
3,50018587,Not Yet Graded,2.0
4,50018565,Not Yet Graded,0.0


Il faut maintenant transformer ce `DataFrame` en faisant l'*inverse* de la fonction `melt()`. Pour cela, nous disposons de la fonction `pivot()` à partir du `DataFrame`. Cette fonction prend trois paramètre :

- `index` : identifiant de chaque ligne (ici, l'identifiant des restaurants)
- `columns` : noms des colonnes (ici, chaque grade) 
- `value` : valeurs des cellules (ici, score moyen pour un restaurant pour un grade)

In [22]:
dfbis = df.pivot(index = "restaurant_id", columns = "grade", values = "score")
dfbis.head()

grade,A,B,C,Not Yet Graded,P,Z
restaurant_id,,,,,,
30075445,6.750000,14.0,NaN,NaN,NaN,NaN
30112340,10.666667,23.0,NaN,NaN,NaN,NaN
30191841,9.250000,NaN,NaN,NaN,NaN,NaN
40356018,9.000000,NaN,NaN,NaN,NaN,NaN
40356068,13.000000,25.0,NaN,NaN,NaN,20.0


Au final, nous regardons les informations de notre `DataFrame` ainsi créé. Nous remarquons que la plupart ont eu un grade `A`, et plusieurs n'ont pas encore été évalué.

In [23]:
dfbis.shape

(24621, 6)

In [24]:
dfbis.count()

grade
A                 23440
B                  8280
C                  2708
Not Yet Graded      512
P                  1154
Z                  1337
dtype: int64

Nous récupérons ici les informations des restaurants (nom - `name` - et quartier - `borough`) pour les ajouter à notre table précédente. Ici, nous récupérons juste les informations que l'on souhaite.

In [27]:
infos = db.restaurants.find({}, { "_id": 0, "restaurant_id": 1, "name": 1, "borough": 1 })
infos_df = pd.DataFrame(list(infos))

Pour réaliser une jointure, nous avons la fonction `join()` qui réalise une jointure sur les `index` des lignes par défaut. La fonction `set_index()` permet de définir cet `index` de lignes pour les informations des restaurants, avant de faire la jointure.

In [28]:
final = dfbis.join(infos_df.set_index("restaurant_id"))
final.head()

,A,B,C,Not Yet Graded,P,Z,borough,name
restaurant_id,,,,,,,,
30075445,6.750000,14.0,NaN,NaN,NaN,NaN,Bronx,Morris Park Bake Shop
30112340,10.666667,23.0,NaN,NaN,NaN,NaN,Brooklyn,Wendy'S
30191841,9.250000,NaN,NaN,NaN,NaN,NaN,Manhattan,Dj Reynolds Pub And Restaurant
40356018,9.000000,NaN,NaN,NaN,NaN,NaN,Brooklyn,Riviera Caterer
40356068,13.000000,25.0,NaN,NaN,NaN,20.0,Queens,Tov Kosher Kitchen


Avec un `DataFrame` ainsi créé, il est maintenant possible, par exemple, de calculer les scores moyens de chaque grade, pour chaque quartier (ici, dans `python`).

In [29]:
final.groupby("borough").mean()

,A,B,C,Not Yet Graded,P,Z
borough,,,,,,
Bronx,8.837432,19.789996,28.903614,14.046512,5.669065,22.376238
Brooklyn,8.941166,19.869812,28.732290,16.412587,5.353474,23.449123
Manhattan,8.935430,19.848022,31.182162,17.836478,5.680101,24.492228
Missing,8.410000,22.000000,43.500000,NaN,NaN,NaN
Queens,9.117012,19.879747,31.122008,14.223881,5.283582,23.509259
Staten Island,9.331491,19.365970,32.820000,12.848485,6.684211,19.770833
